In [1]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)
import os
import sys
project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Go up one level
sys.path.append(project_path)
import pandas as pd
from numpy.random import random
from utils.chatbot import QWENChatBot
from pandarallel import pandarallel
import json
import time
import random
from collections import Counter
import jieba  # For Chinese text segmentation
from wordcloud import WordCloud
import matplotlib.pyplot as plt

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
data_tagged = pd.read_excel('../processed_data/03_tagged_processed_cleaned.xlsx')
import ast
col_list = ['Style_Category', 'Functionality_Category', 'Element_Category',
       'Product_Category', 'Competitor_Category','Product_Mention','Competitor','Keywords_Design_Elements', 'Keywords_Design_Style',
       'Keywords_Functionality']
for col in col_list:

# Convert string representations of lists to actual Python lists
    data_tagged[col] = data_tagged[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
data_tagged

,内容,raw_tagging,parsed_json,Age_Range,Gender,City_Tier,Purchase_Purpose,Purchase_Situation,Product_Mention,Competitor,...,Past_Purchase,Tone,Purpose_Category,Situation_Category,Emotion_Category,Style_Category,Functionality_Category,Element_Category,Product_Category,Competitor_Category
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""...","{'Age_Range': '18-24岁', 'Gender': '女性', 'City_...",18-24岁,女性,NaN,NaN,NaN,[],[],...,否,活泼俏皮,未明确提及,未明确提及,未明确提及,[未明确提及],[未明确提及],[未明确提及],[未明确提及],[未明确提及]
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": ""25-34岁"",\n ""Gender"": ""...","{'Age_Range': '25-34岁', 'Gender': '女性', 'City_...",25-34岁,女性,一线,个人喜好/收藏,NaN,[LV水貂帽衫背心],"[miumiu, Chanel]",...,是,成熟稳重,收藏&投资,未明确提及,喜欢,[未明确提及],[未明确提及],[其他],[accessories],"[MIUMIU, CHANEL]"
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""...","{'Age_Range': '18-24岁', 'Gender': '女性', 'City_...",18-24岁,女性,NaN,打卡/拍照,NaN,[],[],...,否,活泼俏皮,社交需求,未明确提及,喜欢,"[简洁, 其他]",[未明确提及],[未明确提及],[未明确提及],[未明确提及]
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"[黑三彩Carryall, 黑武士羊皮carryall]",[],...,是,成熟稳重,设计感,未明确提及,喜欢,[未明确提及],"[小巧, 斜挎]","[黑三彩, 黑色]",[carryall],[未明确提及]
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n...","{'Age_Range': '', 'Gender': '女性', 'City_Tier':...",NaN,女性,NaN,追求独特设计,NaN,"[卡包, 钱包]",[],...,否,活泼俏皮,设计感,未明确提及,喜欢,[可爱],[实用],"[黑三彩, 白三彩]",[small leather goods],[未明确提及]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,陪伴购物,逛街,[trainer],[],...,否,活泼俏皮,其他,逛街,喜欢,[未明确提及],[未明确提及],[未明确提及],[accessories],[未明确提及]
2623,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"[村上隆太阳花熊猫, 手袋]",[],...,否,活泼俏皮,设计感,未明确提及,喜欢,[其他],[未明确提及],"[太阳花, 其他]","[speedy, carryall]",[未明确提及]
2624,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,NaN,NaN,[],[],...,否,活泼俏皮,未明确提及,未明确提及,满意,[未明确提及],[未明确提及],[未明确提及],[未明确提及],[未明确提及]
2625,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ...","{'Age_Range': '', 'Gender': '', 'City_Tier': '...",NaN,NaN,NaN,追求独特设计,NaN,"[speedy系列, speedynano]",[],...,否,成熟稳重,设计感,未明确提及,满意,[时尚],[未明确提及],"[其他, 老花]",[speedy],[未明确提及]


In [5]:
col_seg = ['Age_Range','City_Tier','Gender','Purchase_Purpose', 'Purchase_Situation', 'Emotion','Product_Mention',
       'Competitor', 'Keywords_Design_Elements', 'Keywords_Design_Style',
       'Keywords_Functionality','Past_Purchase']
df_seg = data_tagged[col_seg]
df_seg

,Age_Range,City_Tier,Gender,Purchase_Purpose,Purchase_Situation,Emotion,Product_Mention,Competitor,Keywords_Design_Elements,Keywords_Design_Style,Keywords_Functionality,Past_Purchase
0,18-24岁,NaN,女性,NaN,NaN,NaN,[],[],[],[],[],否
1,25-34岁,一线,女性,个人喜好/收藏,NaN,喜欢,[LV水貂帽衫背心],"[miumiu, Chanel]",[水貂帽衫背心],[],[],是
2,18-24岁,NaN,女性,打卡/拍照,NaN,喜欢,[],[],[],"[黑白三彩店, 咖啡店]",[],否
3,NaN,NaN,NaN,追求独特设计,NaN,喜欢,"[黑三彩Carryall, 黑武士羊皮carryall]",[],"[黑三彩, 黑武士羊皮]",[],"[单肩斜挎, 不压身高不显大]",是
4,NaN,NaN,女性,追求独特设计,NaN,喜欢,"[卡包, 钱包]",[],"[白三彩, 黑三彩]",[可爱],[实用],否
...,...,...,...,...,...,...,...,...,...,...,...,...
2622,NaN,NaN,NaN,陪伴购物,逛街,开心,[trainer],[],[],[],[],否
2623,NaN,NaN,NaN,追求独特设计,NaN,喜欢,"[村上隆太阳花熊猫, 手袋]",[],"[太阳花, 熊猫]",[合适],[],否
2624,NaN,NaN,NaN,NaN,NaN,满意,[],[],[],[],[],否
2625,NaN,NaN,NaN,追求独特设计,NaN,满意,"[speedy系列, speedynano]",[],"[speedy30小, 上身偏大]",[酷],[],否


In [16]:

def generate_persona(row):
    input = row.to_json(force_ascii=False)
    try:
        response = chatbot.ask(input)  # Ensure chatbot.ask() is functional
        time.sleep(random.uniform(0.2, 0.8))
        return response  # Return the AI's response directly
    except Exception as e:
        # Handle errors gracefully
        return {"Persona_Name": "Error", "Persona_Description": str(e)}

system_message = """
你是一名数据标注专家，现在需要对用户发布的关于 LVMH 和村上隆联名系列的小红书帖子进行分类。请根据以下规则，将用户分为不超过5种类型，并总结每种类型人群的特点。

输出要求：
用户分类（Segmentation）：例如：“潮流先锋”、“奢侈品爱好者”；类型名称应通俗易懂，清晰明了，每类类型不能高度重合，各个分类须体现差异化特点，例如购买动机/场景/风格偏好等。
类型描述（Segmentation_Description）：围绕用户的 Persona，通过具体化的语言描述该人群的核心兴趣或行为特点（如对品牌热爱、艺术感知力、消费场景等），请结合具体输入的信息给出描述，例如：“35岁的一线城市白领，多年前购买过三彩包包，可能会认为新出的村上隆系列有收藏价值。在购买奢侈品的时候更在乎产品设计风格，设计师的理念传递，和物品带来的情绪价值”。
分类理由（Reason）：清晰概括该用户分类所依据的显著标签或特征，列出你认为与其他类型明显的差异化点，包括人口属性/购买动机/行为特征/产品偏好。

注意：
每类类型不能高度重合，各个分类须体现差异化特点。
务必严格遵守类型数量限制，不得超过5种。

输出格式：
{
    "segmentation": "类型名称",
    "description": "该类型人群的特点描述",
    "reason": "分类理由"}
  }

要求：
严格按给定的json格式返回结果，不要添加其他内容
"""
chatbot = QWENChatBot(system_message = system_message)
df_seg = data_tagged[col_seg]
sample = df_seg.sample(n=300, random_state=42)  # Ensure consistent random sampling
#results =  sample.parallel_apply(generate_persona, axis=1)
results = generate_persona(sample)
results

'{\n    "segmentation": "潮流先锋",\n    "description": "25-34岁的一二线城市年轻女性，对时尚潮流有着敏锐的嗅觉和独到的见解。她们喜欢追求独特设计、限量款和联名系列，热衷于在社交媒体上分享自己的穿搭心得，善于利用配饰展现个人风格。对于LV与村上隆的合作系列，她们更看重其设计的独特性和艺术性，愿意为此投入。",\n    "reason": "年龄集中在25-34岁，主要居住在一二线城市；偏好追求独特设计和限量款式；注重产品外观的设计感及艺术价值；常提到‘白三彩’等具体商品以及‘限量’、‘独特’等关键词；购买动机通常为日常使用或社交展示。"\n},\n{\n    "segmentation": "收藏家",\n    "description": "45岁以上的资深奢侈品爱好者，尤其男性居多，他们对品牌历史和经典款式怀有深厚的感情，倾向于收藏具有纪念意义的商品。在选择LVMH与村上隆合作款时，更多是出于情怀和纪念目的，重视产品的收藏价值而非实用性。",\n    "reason": "年龄层偏高，以45岁以上为主；性别比例中男性占比较大；购买动机涉及怀旧/收藏；关注产品背后的故事及其长期保存的价值；偏好经典的黑白三彩系列，如speedy 30公分等经典包型。"\n},\n{\n    "segmentation": "实用主义者",\n    "description": "18-24岁的学生或初入职场的年轻人，他们更加注重商品的功能性和性价比，虽然也关注品牌形象，但首要考量的是物品是否实用、耐用，能否满足日常生活需求。这类用户会仔细评估包袋的容量、材质等因素，在挑选LVMH与村上隆联名款时，也会考虑其实用性。",\n    "reason": "年龄较小，主要为18-24岁区间；可能处于求学阶段或刚步入社会；购买场景多为日常出街、学习生活；强调功能性和实用性；经常提到‘能装’、‘百搭’等特性；提及特定款式如nano speedy等。"\n},\n{\n    "segmentation": "品质享乐派",\n    "description": "35-44岁的都市白领阶层，经济条件较好，购物时既看中商品本身的质量也享受购物过程带来的愉悦感。他们认为购买奢侈品不仅是一种物质消费，更是自我奖励的方式之一。选购LV

In [19]:

def generate_persona(row):
    input = row.to_json(force_ascii=False)
    try:
        response = chatbot.ask(input)  # Ensure chatbot.ask() is functional
        time.sleep(random.uniform(0.2, 0.8))
        return response  # Return the AI's response directly
    except Exception as e:
        # Handle errors gracefully
        return {"Persona_Name": "Error", "Persona_Description": str(e)}
system_message = """

你是一名用户分类专家，现在需要根据用户的行为模式、兴趣偏好以及消费习惯，将用户分为以下六种类型之一。请根据用户提供的信息，选择最合适的分类并进行说明。以下是分类规则：

分类标准：

1. **潮流先锋**:
   - **特点**：25-34岁的女性用户，主要来自一线城市。她们追求独特设计和限量款商品，经常在日常出街或参加活动时使用这些单品，展示自己的时尚品味。
   - **兴趣点**：对品牌与艺术家的合作系列有很高的兴趣，新奇、独特的设计元素，如限量联名款时尚单品。
   - **购买原因**：购买目的强调独特设计和限量版，愿意为前卫且功能性强的联名时尚单品买单。

2. **高端需求消费者**:
   - **特点**：35-44岁的用户，主要分布在一线城市。注重品牌历史和经典设计，倾向于购买作为礼物或收藏的高端投资品。
   - **兴趣点**：品牌的文化传承、设计师理念、经典的情怀和商品的纪念意义。
   - **购买原因**：倾向于高价位的收藏品或限量版，满足收藏价值与个人品味。

3. **实用主义者**:
   - **特点**：18-24岁的用户，包括学生和年轻白领，主要来自一二线城市，关注产品的实用性和性价比。
   - **兴趣点**：商品的容量、背法多样性、搭配方便，以满足日常使用需求。
   - **购买原因**：选择功能性较强的商品如包袋和配件，实用性高且价格中立。

4. **复古怀旧派**:
   - **特点**：45岁以上的用户，分布较广，但以一线城市为主。偏好经典复古款式，尤其是有年代感或设计稀缺的商品。
   - **兴趣点**：反映时代特征的经典产品，如千禧年风格包包、村上隆系列限量版、老花麻将包。
   - **购买原因**：看重商品背后的文化和艺术价值，用作收藏、投资或作为传承物。

5. **无法确定**
   - 其他无法分类的情况

请根据上面的分类标准，对每位用户进行明确分类，并输出对应类型名称。不要添加推理过程/解释或多余文字
"""
chatbot = QWENChatBot(system_message = system_message)
#sample = df_seg.sample(n=300, random_state=42)  # Ensure consistent random sampling
df_seg = data_tagged[col_seg]
df_seg['persona'] =  df_seg.parallel_apply(lambda x: generate_persona(x), axis=1)
df_seg

/var/folders/vx/fds6ny9x1_dfqfcmdl61lsmw0000gp/T/ipykernel_43483/4278224265.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['persona'] =  df_seg.parallel_apply(lambda x: generate_persona(x), axis=1)


,Age_Range,City_Tier,Gender,Purchase_Purpose,Purchase_Situation,Emotion,Product_Mention,Competitor,Keywords_Design_Elements,Keywords_Design_Style,Keywords_Functionality,Past_Purchase,persona
0,18-24岁,NaN,女性,NaN,NaN,NaN,[],[],[],[],[],否,实用主义者
1,25-34岁,一线,女性,个人喜好/收藏,NaN,喜欢,[LV水貂帽衫背心],"[miumiu, Chanel]",[水貂帽衫背心],[],[],是,潮流先锋
2,18-24岁,NaN,女性,打卡/拍照,NaN,喜欢,[],[],[],"[黑白三彩店, 咖啡店]",[],否,实用主义者
3,NaN,NaN,NaN,追求独特设计,NaN,喜欢,"[黑三彩Carryall, 黑武士羊皮carryall]",[],"[黑三彩, 黑武士羊皮]",[],"[单肩斜挎, 不压身高不显大]",是,无法确定
4,NaN,NaN,女性,追求独特设计,NaN,喜欢,"[卡包, 钱包]",[],"[白三彩, 黑三彩]",[可爱],[实用],否,潮流先锋
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,NaN,NaN,NaN,陪伴购物,逛街,开心,[trainer],[],[],[],[],否,无法确定
2623,NaN,NaN,NaN,追求独特设计,NaN,喜欢,"[村上隆太阳花熊猫, 手袋]",[],"[太阳花, 熊猫]",[合适],[],否,潮流先锋
2624,NaN,NaN,NaN,NaN,NaN,满意,[],[],[],[],[],否,无法确定
2625,NaN,NaN,NaN,追求独特设计,NaN,满意,"[speedy系列, speedynano]",[],"[speedy30小, 上身偏大]",[酷],[],否,潮流先锋


In [23]:
data_tagged['persona'] = df_seg['persona']
data_tagged.to_excel('../processed_data/04_segmentation.xlsx',index=False)

In [52]:
col_list2 = ['Age_Range', 'Gender', 'City_Tier','Tone', 'Purchase_Purpose', 'Purchase_Situation',
       'Style_Category', 'Functionality_Category', 'Element_Category',
       'Product_Mention','persona']
df_seg2 = data_tagged[col_list2]

In [83]:
def generate_persona(row):
    input = row.to_json(force_ascii=False)
    try:
        response = chatbot.ask(input)  # Ensure chatbot.ask() is functional
        return response  # Return the AI's response directly
    except Exception as e:
        # Handle errors gracefully
        return {"Persona_Name": "Error", "Persona_Description": str(e)}
system_message = """

你是一名用户分类专家，现在需要根据给定的四类lvmh用户画像，按格式提炼并通过合理的推测补充每个用户画像的信息：
用户画像描述如下：

1. **潮流先锋**:
   - **特点**：25-34岁的女性用户，主要来自一线城市。她们追求独特设计和限量款商品，经常在日常出街或参加活动时使用这些单品，展示自己的时尚品味。
   - **兴趣点**：对品牌与艺术家的合作系列有很高的兴趣，新奇、独特的设计元素，如限量联名款时尚单品。
   - **购买原因**：购买目的强调独特设计和限量版，愿意为前卫且功能性强的联名时尚单品买单。

2. **高端需求消费者**:
   - **特点**：35-44岁的用户，主要分布在一线城市。注重品牌历史和经典设计，倾向于购买作为礼物或收藏的高端投资品。
   - **兴趣点**：品牌的文化传承、设计师理念、经典的情怀和商品的纪念意义。
   - **购买原因**：倾向于高价位的收藏品或限量版，满足收藏价值与个人品味。

3. **实用主义者**:
   - **特点**：18-24岁的用户，包括学生和年轻白领，主要来自一二线城市，关注产品的实用性和性价比。
   - **兴趣点**：商品的容量、背法多样性、搭配方便，以满足日常使用需求。
   - **购买原因**：选择功能性较强的商品如包袋和配件，实用性高且价格中立。

4. **复古怀旧派**:
   - **特点**：45岁以上的用户，分布较广，但以一线城市为主。偏好经典复古款式，尤其是有年代感或设计稀缺的商品。
   - **兴趣点**：反映时代特征的经典产品，如千禧年风格包包、村上隆系列限量版、老花麻将包。
   - **购买原因**：看重商品背后的文化和艺术价值，用作收藏、投资或作为传承物。'''


信息维度如下：
1.年龄层
2.城市等级
3.画像特点描述：详细描述用户画像
4.消费场合：根据你对用户的了解，参考Purchase_Situation，推测用户在什么场合会用到lv的产品，例如旅游，参加重要活动，日常通勤等，需要给出具体的使用场景。
5.产品偏好：根据你对用户的了解以及Product_Mention信息，推测用户感兴趣的3个lvmh和村上隆联名产品，例如carryall，speedy，capucines，旅行箱等，不同用户的偏好产品需要有区分度。
6.一般产品偏好：根据你对用户的了解，推测用户感兴趣的3个lvmh产品，不同用户的偏好产品需要有区分度。
7.价格偏好：根据以上用户画像，推测用户能接受的lvmh产品价格范围，例如<10000,30000-50000等。
8.沟通关键词: 根据你对用户的了解，请概括一些最能打动这类用户的关键词，以便于新产品推广，例如限量联名，经典设计等。
9.对话风格：根据你对用户的了解，请用一些词概括对方的说话语气，例如成熟稳重，活泼俏皮，优雅自信等，必须返回形容词。

要求：不同persona间的维度信息需要有区分度

输出格式：
{
"persona a": {
"Age_Range": str,
"City_Tier": str,
"Description":str,
"Purchase_Occasion": [], 
"Product_Preference": [],
"Product_Preference_General": [],
"Price_Preference": str,
"Keywords_Communication": [], 
"Tone": []}
}

请严格按照格式输出用户画像信息，不要添加多余内容
"""
chatbot = QWENChatBot(system_message = system_message)

sample = df_seg2.sample(n=300, random_state=42) 
persona_dimension = generate_persona(sample)
persona_dimension

'{\n  "潮流先锋": {\n    "Age_Range": "25-34岁",\n    "City_Tier": "一线城市",\n    "Description": "追求独特设计和限量款商品，经常在日常出街或参加活动时使用这些单品，展示自己的时尚品味。",\n    "Purchase_Occasion": ["日常出街", "参加活动", "旅行购物"],\n    "Product_Preference": ["白三彩Speedy Nano", "黑三彩Speedy 30", "LV村上隆联名系列"],\n    "Product_Preference_General": ["Speedy", "Capucines", "On The Go"],\n    "Price_Preference": "<10000 - 30000",\n    "Keywords_Communication": ["限量联名", "独特设计", "时尚潮流"],\n    "Tone": ["活泼俏皮", "兴奋", "成熟稳重"]\n  },\n  "高端需求消费者": {\n    "Age_Range": "35-44岁",\n    "City_Tier": "一线城市",\n    "Description": "注重品牌历史和经典设计，倾向于购买作为礼物或收藏的高端投资品。看重品牌的传承和设计师的理念。",\n    "Purchase_Occasion": ["重要会议", "商务宴请", "家庭纪念日"],\n    "Product_Preference": ["Keepall 行李袋", "经典款手提包", "LV Trainer 白三彩"],\n    "Product_Preference_General": ["Twist", "Neverfull", "Alma"],\n    "Price_Preference": "30000 - 50000",\n    "Keywords_Communication": ["文化遗产", "经典设计", "传承价值"],\n    "Tone": ["成熟稳重", "优雅自信", "怀旧情感"]\n  },\n  "实用主义者": {\n    "Age_Range": "18-24岁",\n  

In [84]:
pd.DataFrame(to_json(persona_dimension))#.to_excel('../persona_description.xlsx')

,潮流先锋,高端需求消费者,实用主义者,复古怀旧派
Age_Range,25-34岁,35-44岁,18-24岁,45岁以上
City_Tier,一线城市,一线城市,一二线城市,一线城市为主
Description,追求独特设计和限量款商品，经常在日常出街或参加活动时使用这些单品，展示自己的时尚品味。,注重品牌历史和经典设计，倾向于购买作为礼物或收藏的高端投资品。看重品牌的传承和设计师的理念。,关注产品的实用性和性价比，选择功能性较强的商品如包袋和配件，以满足日常使用需求。,偏好经典复古款式，尤其是有年代感或设计稀缺的商品。看重商品背后的文化和艺术价值，用作收藏、投...
Purchase_Occasion,"[日常出街, 参加活动, 旅行购物]","[重要会议, 商务宴请, 家庭纪念日]","[日常通勤, 节假日旅游, 校园生活]","[重要庆典, 家庭聚会, 博物馆参观]"
Product_Preference,"[白三彩Speedy Nano, 黑三彩Speedy 30, LV村上隆联名系列]","[Keepall 行李袋, 经典款手提包, LV Trainer 白三彩]","[Vanity PM 纸牌系列, Nano Speedy, All in BB 水桶包]","[老花麻将包, 黑三彩Speedy, 村上隆合作款]"
Product_Preference_General,"[Speedy, Capucines, On The Go]","[Twist, Neverfull, Alma]","[Pochette Accessoires, Petit Noe, NéoNoé]","[Speedy Bandouliere, Lockme, Bumbag]"
Price_Preference,<10000 - 30000,30000 - 50000,<10000,30000 - 50000
Keywords_Communication,"[限量联名, 独特设计, 时尚潮流]","[文化遗产, 经典设计, 传承价值]","[高性价比, 实用功能, 多功能性]","[经典复古, 文化价值, 珍贵稀缺]"
Tone,"[活泼俏皮, 兴奋, 成熟稳重]","[成熟稳重, 优雅自信, 怀旧情感]","[轻松愉快, 活泼俏皮, 信息传递]","[成熟稳重, 怀旧情感, 怀旧与收藏]"
